- The ipyparallel controller creates two files which contain information how to connect to the controller (address and authentication secrets).
- To to connect our engines and clients to the cluster, we need to copy those files to our local machine.
- We use `rsync`.

The address changes, so replace `1.2.3.4` by the address shown on the projector.

In [1]:
!rsync -v 'student@172.25.224.50:.ipython/profile_default/security/ipcontroller-*.json' ~/.ipython/profile_default/security/

The authenticity of host '172.25.224.50 (172.25.224.50)' can't be established.
ED25519 key fingerprint is SHA256:UPJArdVFh2z+vqTUJNA9s4/SqHydNQe59DQnT0wX0zc.
This key is not known by any other names
Are you sure you want to continue connecting (yes/no/[fingerprint])? ^C


Those files are just JSON. We can print them with syntax highlighting with `pygmentize`:

In [7]:
!pygmentize ~/.ipython/profile_default/security/ipcontroller-engine.json 

{
  "ssh": "",
  "interface": "tcp://127.0.0.1",
  "registration": 36329,
  "control": 35269,
  "mux": 41693,
  "task": 46365,
  "iopub": 42197,
  "hb_ping": 33049,
  "hb_pong": 35829,
  "broadcast": [
    35669,
    51201
  ],
  "key": "0b33925e-8a8f32fec28def32726bf6a5",
  "curve_serverkey": null,
  "location": "krowka",
  "pack": "json",
  "unpack": "json",
  "signature_scheme": "hmac-sha256"
}


In [8]:
!pygmentize ~/.ipython/profile_default/security/ipcontroller-client.json 

{
  "ssh": "",
  "interface": "tcp://127.0.0.1",
  "registration": 36329,
  "control": 38189,
  "mux": 44557,
  "task": 49299,
  "task_scheme": "leastload",
  "iopub": 45567,
  "notification": 57511,
  "broadcast": 38491,
  "key": "0b33925e-8a8f32fec28def32726bf6a5",
  "curve_serverkey": null,
  "location": "krowka",
  "pack": "json",
  "unpack": "json",
  "signature_scheme": "hmac-sha256"
}


This starts `htop` in a terminal window to monitor running processes.

If you prefer, start `htop` manually in some terminal window.

In [9]:
!bash -c 'urxvt -e htop & sleep 3'

urxvt: unable to calculate font width for ':slant=0:weight=200:pixelsize=13:minspace=True:lang=he', using max_advance_width.


Now we start our ipcluster engines on the local machine.
They will connect to the controller configured by the JSON `ipcontroller-engine.json`. 

In [13]:
!bash -c 'urxvt -e ipcluster engines & sleep 3'

In [14]:
import ipyparallel as ipp
rc = ipp.Client()

In [15]:
rc.ids

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57]

In [16]:
def hostname():
    import socket
    return socket.gethostname()

In [17]:
rc[:].apply_sync(hostname)

['krowka',
 'krowka',
 'krowka',
 'krowka',
 'aspp07',
 'aspp07',
 'aspp07',
 'aspp07',
 'aspp02',
 'aspp02',
 'aspp02',
 'aspp02',
 'aspp15',
 'aspp15',
 'aspp15',
 'aspp15',
 'aspp16',
 'aspp16',
 'aspp16',
 'aspp16',
 'aspp01',
 'aspp01',
 'aspp01',
 'aspp01',
 'aspp04',
 'aspp04',
 'aspp04',
 'aspp04',
 'aspp12',
 'aspp12',
 'aspp12',
 'aspp12',
 'aspp10',
 'aspp10',
 'aspp05',
 'aspp05',
 'aspp05',
 'aspp05',
 'aspp08',
 'aspp08',
 'aspp08',
 'aspp08',
 'aspp03',
 'aspp03',
 'aspp03',
 'aspp03',
 'aspp05',
 'aspp05',
 'aspp05',
 'aspp05',
 'aspp05',
 'aspp05',
 'aspp05',
 'aspp05',
 'aspp14',
 'aspp14',
 'aspp14',
 'aspp14']

In [18]:
def f(x):
    return x**4 - 3*x

In [20]:
def integrate(a, b, n):
    dx = (b - a) / n
    s = 0.0
    x = a + 0.5 * dx
    for i in range(n):
        y = f(x)
        s += y
        x += dx
    return s * dx





In [19]:
rc[:].push(dict(f=f)).wait()

True

In [24]:
stepcounts = list(range(10_000, 20_000, 1000))
a = [-2 for _ in stepcounts]
b = [+2 for _ in stepcounts]

In [25]:
%timeit -r 1 -n 1    list(map(integrate, a, b, stepcounts))

34.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [26]:
%timeit -r 1 -n 1    rc[:].map_sync(integrate, a, b, stepcounts)

10.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
